Download a collection of past buoy transmissions from a section of the IABP website.

In [1]:
import os
import pandas as pd
import requests

# URL of the file to download
file_url = 'https://iabp.apl.uw.edu/Data_Products/LEVEL1_DATA/LEVEL1_2023.csv'

# Directory to save the downloaded file
temp_dir = '../data/raw/buoydata/past/temp'
os.makedirs(temp_dir, exist_ok=True)

# Path to save the downloaded file
temp_file_path = os.path.join(temp_dir, 'IABP_Level1_2023all.csv')

# Download the file
response = requests.get(file_url)
response.raise_for_status()  # Check if the request was successful

# Save the file
with open(temp_file_path, 'wb') as file:
    file.write(response.content)

print(f'Downloaded file to {temp_file_path}')

# Read the downloaded CSV file
df = pd.read_csv(temp_file_path)

# Directory to save the separated CSV files
output_dir = '../data/raw/buoydata/past'
os.makedirs(output_dir, exist_ok=True)

# Clear all files in the directory before saving new data, except the "temp" folder
for filename in os.listdir(output_dir):
    file_path = os.path.join(output_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path) and filename != 'temp':
            os.rmdir(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')
    
# Separate the file into discrete CSV files based on the buoyID column
for buoy_id, group in df.groupby('BuoyID'):
    output_file_path = os.path.join(output_dir, f'{buoy_id}.csv')
    group.to_csv(output_file_path, index=False)
    print(f'Saved {output_file_path}')

Downloaded file to ../data/raw/buoydata/past/temp\IABP_Level1_2023all.csv
Saved ../data/raw/buoydata/past\900115.csv
Saved ../data/raw/buoydata/past\900120.csv
Saved ../data/raw/buoydata/past\900121.csv
Saved ../data/raw/buoydata/past\900122.csv
Saved ../data/raw/buoydata/past\900124.csv
Saved ../data/raw/buoydata/past\900125.csv
Saved ../data/raw/buoydata/past\900126.csv
Saved ../data/raw/buoydata/past\900127.csv
Saved ../data/raw/buoydata/past\900128.csv
Saved ../data/raw/buoydata/past\900130.csv
Saved ../data/raw/buoydata/past\900131.csv
Saved ../data/raw/buoydata/past\900135.csv
Saved ../data/raw/buoydata/past\900136.csv
Saved ../data/raw/buoydata/past\900137.csv
Saved ../data/raw/buoydata/past\900138.csv
Saved ../data/raw/buoydata/past\900139.csv
Saved ../data/raw/buoydata/past\900140.csv
Saved ../data/raw/buoydata/past\900141.csv
Saved ../data/raw/buoydata/past\902002.csv
Saved ../data/raw/buoydata/past\902004.csv
Saved ../data/raw/buoydata/past\902005.csv
Saved ../data/raw/buoyd

This section will download data from the 2023 ERA5 reanalysis  and save as a netCDF. This data will be interpolated with the past buoy data as training data for machine learning algorithms.

In [ ]:
import os
import cdsapi

# Define the output directory
output_dir = "../data/raw/reanalyses/ERA5"
os.makedirs(output_dir, exist_ok=True)

# Set the CDSAPI_RC environment variable to the path of your .cdsapirc file
os.environ['CDSAPI_RC'] = '../.cdsapirc'

# List of variables to download
variables = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "100m_u_component_of_wind",
    "100m_v_component_of_wind",
    "sea_ice_cover"
]

# Loop over variables to download each separately
client = cdsapi.Client()
dataset = "reanalysis-era5-single-levels"

for variable in variables:
    print(f"Downloading {variable}...")

    # Request configuration for the specific variable
    request = {
        "product_type": ["reanalysis"],
        "variable": [variable],
        "year": ["2023"],
        "month": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11", "12"
        ],
        "day": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11", "12",
            "13", "14", "15",
            "16", "17", "18",
            "19", "20", "21",
            "22", "23", "24",
            "25", "26", "27",
            "28", "29", "30",
            "31"
        ],
        "time": [
            "00:00", "01:00", "02:00",
            "03:00", "04:00", "05:00",
            "06:00", "07:00", "08:00",
            "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00",
            "15:00", "16:00", "17:00",
            "18:00", "19:00", "20:00",
            "21:00", "22:00", "23:00"
        ],
        "data_format": "netcdf",
        "area": [90, -180, 60, 180]  # North, West, South, East
    }

    # Define the target file path
    target_file = os.path.join(output_dir, f"era5_{variable}_2023.nc")
    
    # Retrieve the data
    client.retrieve(dataset, request, target_file)

    print(f"{variable} downloaded successfully to {target_file}.")

2024-12-08 16:51:13,544 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-08 16:51:13,544 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-12-08 16:51:13,545 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-12-08 16:51:13,545 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using t

2024-12-08 16:51:14,258 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-12-08 16:51:14,259 INFO Request ID is 09921051-b4bc-419b-ac60-0f6f50b1c128
2024-12-08 16:51:14,566 INFO status has been updated to accepted
2024-12-08 16:51:28,899 INFO status has been updated to running
2024-12-08 17:11:47,635 INFO status has been updated to successful


5a4b0ed52bd1cc7fd7c299909c0af492.nc:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

10m_u_component_of_wind downloaded successfully to ../data/raw/reanalyses/ERA5\era5_10m_u_component_of_wind_2023.nc.


2024-12-08 17:19:12,263 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-12-08 17:19:12,264 INFO Request ID is c33e10ec-2069-46b9-9723-045dba4de190
2024-12-08 17:19:12,546 INFO status has been updated to accepted
2024-12-08 17:19:18,028 INFO status has been updated to running
2024-12-08 17:39:48,051 INFO status has been updated to successful


...

59da1a6a640dd7b95d16f9ebe21e4e4b.nc:   0%|          | 0.00/31.1M [00:00<?, ?B/s]

model_bathymetry downloaded successfully to ../data/raw/reanalyses/ERA5\era5_model_bathymetry_2023.nc.


2024-12-08 20:02:46,236 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-12-08 20:02:46,237 INFO Request ID is 7afacda2-2b5b-4a55-8833-58ea49ad97f7
2024-12-08 20:02:46,469 INFO status has been updated to accepted
2024-12-08 20:02:52,004 INFO status has been updated to running
2024-12-08 20:21:17,091 INFO status has been updated to successful


a842e0234f28efa39a28e99d9921e36d.nc:   0%|          | 0.00/151M [00:00<?, ?B/s]

sea_ice_cover downloaded successfully to ../data/raw/reanalyses/ERA5\era5_sea_ice_cover_2023.nc.
